In [1]:
!pip install -qq datasets==2.16.1 evaluate==0.4.1 transformers[sentencepiece]==4.35.2
!pip install -qq accelerate==0.26.1
!apt install git-lfs


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 123.5/123.5 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 507.1/507.1 kB 24.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 48.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 33.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 10.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 270.9/270.9 kB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21.3 MB 15.0 MB/s eta 0:00:00
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
git-lfs is already the newest version (3.0.2-1ubuntu0.2).
0 upgraded, 0 newly installed, 0 to remove and 45 

In [15]:
from huggingface_hub import notebook_login
notebook_login ()


In [3]:
import numpy as np
from tqdm.auto import tqdm
import collections
import torch
from datasets import load_dataset
from transformers import AutoTokenizer
from transformers import AutoModelForQuestionAnswering
from transformers import TrainingArguments
from transformers import Trainer
import evaluate

device = torch.device("cuda") if torch.cuda.is_available () else torch.device("cpu")


/usr/local/lib/python3.10/dist-packages/transformers/utils/generic.py:441: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
/usr/local/lib/python3.10/dist-packages/transformers/utils/generic.py:309: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
/usr/local/lib/python3.10/dist-packages/transformers/utils/generic.py:309: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(


In [4]:
# Tên mô hình checkpoint
MODEL_NAME = "distilbert-base-uncased"

# Độ dài tối đa cho mỗi đoạn văn bản sau khi được xử lý
MAX_LENGTH = 384

# Khoảng cách giữa các điểm bắt đầu của các đoạn văn bản liên tiếp
STRIDE = 128


In [5]:
# Download squad dataset từ HuggingFace
DATASET_NAME = 'squad_v2'
raw_datasets = load_dataset ( DATASET_NAME )

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/130319 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/11873 [00:00<?, ? examples/s]

In [6]:
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [7]:
# Định nghĩa hàm preprocess_training_examples và nhận đối số examples là dữ liệu đào tạo
def preprocess_training_examples(examples):
    # Trích xuất danh sách câu hỏi từ examples và loại bỏ các khoảng trắng dư thừa
    questions = [q.strip() for q in examples["question"]]

    # Tiến hành mã hóa thông tin đầu vào sử dụng tokenizer
    inputs = tokenizer(
        questions,
        examples["context"],
        max_length=MAX_LENGTH,
        truncation="only_second",
        stride=STRIDE,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length",
    )

    # Trích xuất offset_mapping từ inputs và loại bỏ nó ra khỏi inputs
    offset_mapping = inputs.pop("offset_mapping")

    # Trích xuất sample_map từ inputs và loại bỏ nó ra khỏi inputs
    sample_map = inputs.pop("overflow_to_sample_mapping")

    # Trích xuất thông tin về câu trả lời (answers) từ examples
    answers = examples["answers"]

    # Khởi tạo danh sách các vị trí bắt đầu và kết thúc câu trả lời
    start_positions = []
    end_positions = []

    # Duyệt qua danh sách offset_mapping
    for i, offset in enumerate(offset_mapping):
        # Xác định index của mẫu (sample) liên quan đến offset hiện tại
        sample_idx = sample_map[i]

        # Trích xuất sequence_ids từ inputs
        sequence_ids = inputs.sequence_ids(i)

        # Xác định vị trí bắt đầu và kết thúc của ngữ cảnh
        idx = 0
        while sequence_ids[idx] != 1:
            idx += 1
        context_start = idx
        while sequence_ids[idx] == 1:
            idx += 1
        context_end = idx - 1

        # Trích xuất thông tin về câu trả lời cho mẫu này
        answer = answers[sample_idx]

        if len(answer['text']) == 0:
            start_positions.append(0)
            end_positions.append(0)
        else:
            # Xác định vị trí ký tự bắt đầu và kết thúc của câu trả lời trong ngữ cảnh
            start_char = answer["answer_start"][0]
            end_char = answer["answer_start"][0] + len(answer["text"][0])

            # Nếu câu trả lời không nằm hoàn toàn trong ngữ cảnh, gán nhãn là (0, 0)
            if offset[context_start][0] > start_char or offset[context_end][1] < end_char:
                start_positions.append(0)
                end_positions.append(0)
            else:
                # Nếu không, gán vị trí bắt đầu và kết thúc dựa trên vị trí của các mã thông tin
                idx = context_start
                while idx <= context_end and offset[idx][0] <= start_char:
                    idx += 1
                start_positions.append(idx - 1)

                idx = context_end
                while idx >= context_start and offset[idx][1] >= end_char:
                    idx -= 1
                end_positions.append(idx + 1)

    # Thêm thông tin vị trí bắt đầu và kết thúc vào inputs
    inputs["start_positions"] = start_positions
    inputs["end_positions"] = end_positions

    return inputs


In [8]:
train_dataset = raw_datasets["train"].map(
    preprocess_training_examples,
    batched = True,
    remove_columns = raw_datasets["train"].column_names ,
)


Map:   0%|          | 0/130319 [00:00<?, ? examples/s]

In [9]:
# In ra độ dài của tập dữ liệu " train " ban đầu và độ dài của tập dữ liệu đã được xử lý ( train_dataset )
len(raw_datasets["train"]), len(train_dataset)


(130319, 131754)

In [10]:
def preprocess_validation_examples(examples):
    # Chuẩn bị danh sách câu hỏi bằng cách loại bỏ khoảng trắng ở đầu và cuối mỗi câu hỏi
    questions = [q.strip() for q in examples["question"]]

    # Sử dụng tokenizer để mã hóa các câu hỏi và văn bản liên quan
    inputs = tokenizer(
        questions,
        examples["context"],
        max_length=MAX_LENGTH,
        truncation="only_second",
        stride=STRIDE,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length",
    )

    # Lấy ánh xạ để ánh xạ lại ví dụ tham chiếu cho từng dòng trong inputs
    sample_map = inputs.pop("overflow_to_sample_mapping")
    example_ids = []

    # Xác định ví dụ tham chiếu cho mỗi dòng đầu vào và điều chỉnh ánh xạ offset
    for i in range(len(inputs["input_ids"])):
        sample_idx = sample_map[i]
        example_ids.append(examples["id"][sample_idx])

    sequence_ids = inputs.sequence_ids(i)
    offset = inputs["offset_mapping"][i]

    # Loại bỏ các offset không phù hợp với sequence_ids
    inputs["offset_mapping"][i] = [
        o if sequence_ids[k] == 1 else None for k, o in enumerate(offset)
    ]

    # Thêm thông tin ví dụ tham chiếu vào đầu vào
    inputs["example_id"] = example_ids

    return inputs

In [11]:
validation_dataset = raw_datasets["validation"].map(
    preprocess_validation_examples,
    batched=True,
    remove_columns=raw_datasets["validation"].column_names,
)

# In ra độ dài của raw_datasets["validation"] và validation_dataset để so sánh
print(len(raw_datasets["validation"]), len(validation_dataset))

Map:   0%|          | 0/11873 [00:00<?, ? examples/s]

11873 12134


In [12]:
model = AutoModelForQuestionAnswering.from_pretrained(MODEL_NAME)


model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForQuestionAnswering were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['qa_outputs.weight', 'qa_outputs.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [13]:
args = TrainingArguments(
    output_dir ="distilbert-finetuned-squadv2", # Thư mục lưu trữ kết quả huấn luyện
    evaluation_strategy ="no", # Chế độ đánh giá không tự động sau mỗi epoch
    save_strategy ="epoch", # Lưu checkpoint sau mỗi epoch
    learning_rate =2e-5 , # Tốc độ học
    num_train_epochs =3 , # Số epoch huấn luyện
    weight_decay =0.01 , # Giảm trọng lượng mô hình để tránh overfitting
    fp16 =True , # Sử dụng kiểu dữ liệu half - precision (16 - bit) để tối ưu hóa
    push_to_hub =True ,
)




In [16]:
# Khởi tạo một đối tượng Trainer để huấn luyện mô hình
trainer = Trainer (
model =model , # Sử dụng mô hình đã tạo trước đó
args =args , # Các tham số và cấu hình huấn luyện
train_dataset = train_dataset , # Sử dụng tập dữ liệu huấn luyện
eval_dataset = validation_dataset , # Sử dụng tập dữ liệu đánh giá
tokenizer = tokenizer , # Sử dụng tokenizer để xử lý văn bản
)

# Bắt đầu quá trình huấn luyện
trainer . train ()


You're using a DistilBertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss
500,3.085600
1000,2.090600
1500,1.869900
2000,1.755700
2500,1.664100
3000,1.597500
3500,1.545200
4000,1.543700
4500,1.479200
5000,1.457800


TrainOutput(global_step=49410, training_loss=1.011793383729769, metrics={'train_runtime': 6110.0034, 'train_samples_per_second': 64.691, 'train_steps_per_second': 8.087, 'total_flos': 3.873165421863629e+16, 'train_loss': 1.011793383729769, 'epoch': 3.0})

In [17]:
# Gửi dữ liệu đào tạo lên Hub
trainer.push_to_hub(commit_message ="Training complete")


CommitInfo(commit_url='https://huggingface.co/Duchaha/distilbert-finetuned-squadv2/commit/54b110b382ced4bb83f30e1d52f3087989233c79', commit_message='Training complete', commit_description='', oid='54b110b382ced4bb83f30e1d52f3087989233c79', pr_url=None, pr_revision=None, pr_num=None)

In [18]:
# Tải metric " squad " từ thư viện evaluate
metric = evaluate.load ("squad_v2")


In [21]:
N_BEST = 20 # Số lượng kết quả tốt nhất được lựa chọn sau khi dự đoán
MAX_ANS_LENGTH = 30 # Độ dài tối đa cho câu trả lời dự đoán

def compute_metrics(start_logits, end_logits, features, examples):
    # Create a default dictionary to map each example to its corresponding list of features
    example_to_features = collections.defaultdict(list)
    for idx, feature in enumerate(features):
        example_to_features[feature['example_id']].append(idx)

    predicted_answers = []
    for example in tqdm(examples):
        example_id = example['id']
        context = example['context']
        answers = []

        # Iterate through all features related to the example
        for feature_index in example_to_features[example_id]:
            start_logit = start_logits[feature_index]
            end_logit = end_logits[feature_index]
            offsets = features[feature_index]['offset_mapping']

            # Get the indices with the highest values for start and end logits
            start_indexes = np.argsort(start_logit)[-1: -N_BEST - 1: -1].tolist()
            end_indexes = np.argsort(end_logit)[-1: -N_BEST - 1: -1].tolist()

            for start_index in start_indexes:
                for end_index in end_indexes:
                    # Skip answers that do not fall entirely within the context
                    if offsets[start_index] is None or offsets[end_index] is None:
                        continue
                    # Skip answers with length greater than max_answer_length
                    if end_index - start_index + 1 > MAX_ANS_LENGTH:
                        continue

                    # Create a new answer
                    answer = {
                        'text': context[offsets[start_index][0]: offsets[end_index][1]],
                        'logit_score': start_logit[start_index] + end_logit[end_index],
                    }
                    answers.append(answer)

        # Select the answer with the best score
        if len(answers) > 0:
            best_answer = max(answers, key=lambda x: x['logit_score'])
            answer_dict = {
                'id': example_id,
                'prediction_text': best_answer['text'],
                'no_answer_probability': 1 - best_answer['logit_score']
            }
        else:
            answer_dict = {
                'id': example_id,
                'prediction_text': '',
                'no_answer_probability': 1.0
            }
        predicted_answers.append(answer_dict)

    # Create a list of theoretical answers from examples
    theoretical_answers = [
        {'id': ex['id'], 'answers': ex['answers']} for ex in examples
    ]

    # Use metric.compute to calculate metrics and return the results
    return metric.compute(predictions=predicted_answers, references=theoretical_answers)

In [22]:
# Thực hiện dự đoán trên tập dữ liệu validation
predictions , _, _ = trainer.predict ( validation_dataset )
start_logits , end_logits = predictions
results = compute_metrics (
 start_logits ,
 end_logits ,
 validation_dataset ,
 raw_datasets ["validation"]
 )
results


  0%|          | 0/11873 [00:00<?, ?it/s]

{'exact': 66.78177377242483,
 'f1': 69.8554010489253,
 'total': 11873,
 'HasAns_exact': 66.29554655870446,
 'HasAns_f1': 72.45161549492025,
 'HasAns_total': 5928,
 'NoAns_exact': 67.26661059714046,
 'NoAns_f1': 67.26661059714046,
 'NoAns_total': 5945,
 'best_exact': 67.71666807041186,
 'best_exact_thresh': -7.107421875,
 'best_f1': 70.19562476533622,
 'best_f1_thresh': -4.8203125}

In [24]:
# Use a pipeline as a high-level helper
from transformers import pipeline

pipe = pipeline("question-answering", model="Duchaha/distilbert-finetuned-squadv2")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/561 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/utils/generic.py:309: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(


model.safetensors:   0%|          | 0.00/265M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.20k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/712k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

In [26]:
INPUT_QUESTION = 'What is my name ?'
INPUT_CONTEXT = 'My name is AI Vietnam and I live in Vietnam .'
pipe( question = INPUT_QUESTION , context = INPUT_CONTEXT )


{'score': 0.9419191479682922, 'start': 11, 'end': 21, 'answer': 'AI Vietnam'}